In [1]:
# Imports
import itertools
import multiprocessing
from datetime import datetime

import smact
from smact import Element, Species, element_dictionary, neutral_ratios, ordered_elements
from smact.screening import pauling_test, smact_filter

In [2]:
# Generate the dictionary of elements
all_el = element_dictionary(
    elements=ordered_elements(1, 83)
)  # A dictionary of all element objects
symbols_list = list(all_el.keys())
all_els = [all_el[symbol] for symbol in symbols_list]
dont_want = ["He", "Ne", "Ar", "Kr", "Xe", "Pm", "Tc"]

for unwanted in dont_want:
    symbols_list.remove(unwanted)
all_els = [all_el[symbol] for symbol in symbols_list]
coord_els = [el.coord_envs for el in all_els]

In [3]:
# Elements for A

# A requires elements which can take a coordination state of 8
A_els = []
B_els = []

for el in all_els:
    if el.coord_envs == None:
        continue
    CNs = [i.split("_")[0] for i in el.coord_envs]
    if "8" in CNs:
        A_els.append(el)
    if "6" in CNs:
        B_els.append(el)

In [4]:
# Elements for C and D
C_list = ["Li"]
D_list = ["O"]
C_els = [all_el[symbol] for symbol in C_list]
D_els = [all_el[symbol] for symbol in D_list]

In [5]:
print(f"The number of  allowed elements for (A) are: {len(A_els)} \n")
print("The number of allowed elements for {B} are':" f"{len(B_els)} \n")

The number of  allowed elements for (A) are: 45 

The number of allowed elements for {B} are':75 



In [6]:
# Generate the A-B-C-D combinations

ABCD_pairs = [(x, y, z, a) for x in A_els for y in B_els for z in C_els for a in D_els]

# Prove to ourselves that we have all unique chemical systems
print(f"We have generated {len(ABCD_pairs)} potential compounds")

# for i in oxide_systems:
#   print(f"{i[0].symbol} {i[1].symbol} {i[2].symbol} {i[3].symbol}")

We have generated 3375 potential compounds


In [7]:
def smact_filter(els, stoichs=[[3], [2], [3], [12]], species_unique=True):
    """Function that applies the charge neutrality and electronegativity
    tests in one go for simple application in external scripts that
    wish to apply the general 'smact test'.

    Args:
        els (tuple/list): A list of smact.Element objects
        threshold (int): Threshold for stoichiometry limit, default = 8
        species_unique (bool): Whether or not to consider elements in different
        oxidation states as unique in the results.
    Returns:
        allowed_comps (list): Allowed compositions for that chemical system
        in the form [(elements), (oxidation states), (ratios)] if species_unique=True
        or in the form [(elements), (ratios)] if species_unique=False.
    """
    compositions = []

    # Get symbols and electronegativities
    symbols = tuple(e.symbol for e in els)
    electronegs = [e.pauling_eneg for e in els]
    ox_combos = [e.oxidation_states for e in els]
    for ox_states in itertools.product(*ox_combos):
        # Test for charge balance
        cn_e, cn_r = neutral_ratios(ox_states, stoichs=stoichs)
        # Electronegativity test
        if cn_e:
            electroneg_OK = pauling_test(ox_states, electronegs)
            if electroneg_OK:
                for ratio in cn_r:
                    compositions.append(tuple([symbols, ox_states, ratio]))

    # Return list depending on whether we are interested in unique species combinations
    # or just unique element combinations.
    if species_unique:
        return compositions
    else:
        compositions = [(i[0], i[2]) for i in compositions]
        compositions = list(set(compositions))
        return compositions

In [11]:
# Use multiprocessing and smact_filter to quickly generate our list of compositions
start = datetime.now()
if __name__ == "__main__":  # Always use pool protected in an if statement
    with multiprocessing.Pool(24) as p:  # start 4 worker processes
        result = p.map(smact_filter, ABCD_pairs)
print(f"Time taken to generate list:  {datetime.now() - start}")

Time taken to generate list:  0:00:00.175910


In [12]:
# Flatten the list of lists
flat_list = [item for sublist in result for item in sublist]
print(f"Number of compositions: --> {len(flat_list)} <--")
print("Each list entry looks like this:\n  elements, oxidation states, stoichiometries")
for i in flat_list[:5]:
    print(i)

Number of compositions: --> 2504 <--
Each list entry looks like this:
  elements, oxidation states, stoichiometries
(('Li', 'B', 'Li', 'O'), (1, 3, 1, -1), (3, 2, 3, 12))
(('Li', 'C', 'Li', 'O'), (1, 3, 1, -1), (3, 2, 3, 12))
(('Li', 'N', 'Li', 'O'), (1, 3, 1, -1), (3, 2, 3, 12))
(('Li', 'Al', 'Li', 'O'), (1, 3, 1, -1), (3, 2, 3, 12))
(('Li', 'Si', 'Li', 'O'), (1, 3, 1, -1), (3, 2, 3, 12))


In [13]:
from pymatgen import Composition


def comp_maker(comp):
    form = []
    for el, ammt in zip(comp[0], comp[2]):
        form.append(el)
        form.append(ammt)
    form = "".join(str(e) for e in form)
    pmg_form = Composition(form).reduced_formula
    return pmg_form


if __name__ == "__main__":
    with multiprocessing.Pool() as p:
        pretty_formulas = p.map(comp_maker, flat_list)

print("Each list entry now looks like this: ")
for i in pretty_formulas[:5]:
    print(i)

Each list entry now looks like this: 
Li3BO6
Li3CO6
Li3NO6
Li3AlO6
Li3SiO6


In [14]:
# Calculate sustainability
def sus_calc(comp):
    sus_factor = 0
    for i in Composition(comp).elements:
        sus_factor += (
            Composition(comp).get_wt_fraction(i) * smact.Element(i.symbol).HHI_r
        )
    return sus_factor


# Compute sustainability
start = datetime.now()
if __name__ == "__main__":
    with multiprocessing.Pool() as p:
        sus_factors = p.map(sus_calc, pretty_formulas)
# sus_factors=[sus_calc(Composition(i)) for i in pretty_formulas]
print(f"Time taken to calculate sus factors:  {datetime.now()-start}")

Time taken to calculate sus factors:  0:00:01.067193


In [15]:
# We want to be able to feed in a list of species into a structure predictor
# Need to our list in the form of [(A,x), (B,y), (C,z)]
# Where A,B,C are species, dtype-string
# x,y,z are the charges/oxidation state - dtype: int
from smact.structure_prediction.utilities import parse_spec, unparse_spec

species = []
A = []
B = []
C = []
D = []
for i in range(len(flat_list)):
    species.append(pretty_formulas[i])
    A.append(unparse_spec((flat_list[i][0][0], flat_list[i][1][0])))
    B.append(unparse_spec((flat_list[i][0][1], flat_list[i][1][1])))
    C.append(unparse_spec((flat_list[i][0][2], flat_list[i][1][2])))
    D.append(unparse_spec((flat_list[i][0][3], flat_list[i][1][3])))

In [16]:
import pandas as pd

columns = ["Pretty Formula", "A", "B", "C", "D"]
df_list = [[species[i], A[i], B[i], C[i], D[i]] for i in range(len(species))]
df = pd.DataFrame(data=df_list, columns=columns)
df["sus_factor"] = sus_factors
df = df.sort_values(by="sus_factor", ascending=True)
df = df.reset_index(drop=True)

df.head()

,Pretty Formula,A,B,C,D,sus_factor
0,Li3Mg3(NO6)2,Mg1+,N3+,Li1+,O1-,745.566611
1,Na3Li3(NO6)2,Na1+,N3+,Li1+,O1-,748.694226
2,Li3Mg3(CO6)2,Mg1+,C3+,Li1+,O1-,748.731405
3,Na3Li3(CO6)2,Na1+,C3+,Li1+,O1-,751.940682
4,Li3Mg3Al2O12,Mg1+,Al3+,Li1+,O1-,806.236425


In [17]:
df.to_csv("./Li-Garnet_Comps_sus.csv", index=False)
df.head()

,Pretty Formula,A,B,C,D,sus_factor
0,Li3Mg3(NO6)2,Mg1+,N3+,Li1+,O1-,745.566611
1,Na3Li3(NO6)2,Na1+,N3+,Li1+,O1-,748.694226
2,Li3Mg3(CO6)2,Mg1+,C3+,Li1+,O1-,748.731405
3,Na3Li3(CO6)2,Na1+,C3+,Li1+,O1-,751.940682
4,Li3Mg3Al2O12,Mg1+,Al3+,Li1+,O1-,806.236425
